# Basic Example

This notebook demonstrates the basic usage of qTPU for quantum circuit processing.

In [ ]:
import numpy as np
from qiskit.circuit.library import TwoLocal


def simple_circuit(n: int):
    circuit = TwoLocal(n, ["u"], "rzz", entanglement="linear", reps=2).decompose()
    circuit = circuit.assign_parameters(
        {param: np.random.rand() * np.pi / 2 for param in circuit.parameters}
    )
    circuit.measure_all()
    return circuit

circuit = simple_circuit(6)

circuit.draw(output="mpl", fold=-1)

In [ ]:
import qtpu

# cut the circuit into two halves
cut_circ = qtpu.cut(circuit, num_qubits=circuit.num_qubits // 2)

cut_circ.draw(output="mpl", fold=-1)

In [ ]:
# convert the circuit into a hybrid tensor network
hybrid_tn = qtpu.circuit_to_hybrid_tn(cut_circ)

for i, subcirc in enumerate(hybrid_tn.subcircuits):
    print(f"Subcircuit {i}:")
    print(subcirc)
    print("--------------------")

In [ ]:
# evaluate the hybrid tensor network to a classical tensor network
tn = qtpu.evaluate(hybrid_tn)

# contract the classical tensor network
qtpu_res = tn.contract(all, optimize="auto-hq", output_inds=[])

In [ ]:
from qiskit.circuit import QuantumCircuit
from qiskit_aer.primitives import EstimatorV2


def run_comparison(circuit: QuantumCircuit):
    circuit = circuit.remove_final_measurements(inplace=False)
    return (
        EstimatorV2()
        .run([(circuit, "Z" * circuit.num_qubits)], precision=0.0000)
        .result()[0]
        .data.evs
    )

qiskit_res = run_comparison(circuit)

In [ ]:
print(f"QTPU result: {qtpu_res}")
print(f"Qiskit result: {qiskit_res}")
print(f"Error: {round(abs(qtpu_res - qiskit_res), 5)}")